# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
filepath = os.path.join("../output_data/cities.csv")
city_df = pd.read_csv(filepath)
city_df

,Unnamed: 0,City,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloud Cover %,Country
0,0,Punta Arenas,-70.92,-53.15,55.400,43,6.70,40,CL
1,1,Thompson,-97.86,55.74,15.800,47,4.10,40,CA
2,2,New Norfolk,147.06,-42.78,42.008,86,1.09,60,AU
3,3,Usvyaty,30.75,55.75,35.798,86,2.21,100,RU
4,4,Port Elizabeth,25.57,-33.92,66.200,93,2.10,0,ZA
...,...,...,...,...,...,...,...,...,...
527,527,Pampa,-100.96,35.54,71.996,37,9.80,1,US
528,528,Havelock,-76.90,34.88,69.998,49,10.30,75,US
529,529,Sakakah,40.21,29.97,69.800,37,3.10,20,SA
530,530,Musoma,33.80,-1.50,68.036,66,1.83,100,TZ


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

locations = city_df[["Latitude","Longitude"]].astype(float)
humidity = city_df["Humidity"].astype(float)

In [17]:
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig = gmaps.figure()
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# A max temperature lower than 80 degrees but higher than 70.
maxed_temp = city_df [city_df['Temperature']<80]
filtered_temp = maxed_temp[maxed_temp['Temperature']>70]

  # Wind speed less than 10 mph.
wind_filtered_df= filtered_temp[filtered_temp['Wind Speed']<10]
wind_filtered_df
  # Zero cloudiness.
cloud_filtered_df = wind_filtered_df[wind_filtered_df['Cloud Cover %'] == 0]

cloud_filtered_df = cloud_filtered_df.rename(columns={'Longitude':'Lng', 'Latitude': 'Lat'})
  #Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
cloud_filtered_df['Hotel Name'] = ''
hotel_df = cloud_filtered_df
hotel_df

,Unnamed: 0,City,Lng,Lat,Temperature,Humidity,Wind Speed,Cloud Cover %,Country,Hotel Name
38,38,East London,27.91,-33.02,71.222,66,7.84,0,ZA,
80,80,Puerto Rico,-55.02,-26.80,74.372,40,0.52,0,AR,
151,151,Pathein,94.73,16.78,76.190,84,3.55,0,MM,
163,163,Mount Isa,139.50,-20.73,71.600,30,4.10,0,AU,
192,192,Delta del Tigre,-56.39,-34.76,75.992,44,5.10,0,UY,
213,213,Abu Dhabi,54.37,24.47,77.000,73,3.10,0,AE,
289,289,Gualeguay,-59.31,-33.14,74.462,46,2.47,0,AR,
305,305,Atar,-13.05,20.52,74.984,19,3.72,0,MR,
324,324,Hīt,42.83,33.64,75.128,30,1.45,0,IQ,
355,355,Upington,21.26,-28.45,73.400,35,2.60,0,ZA,


In [82]:

base_url = base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}



In [81]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    hotel_site = row['City']

    # add keyword to params dict
    #params['keyword'] = hotel_site
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {hotel_site}.")

    # extract results
    response = requests.get(base_url, params=params).json()
    hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    
    #try:
    #print(f"Closest hotel for {hotel_site} is {['results'][0]['name']}.")
    #except:
        #print('Something went wrong'


Retrieving Results for Index 38: East London.
Retrieving Results for Index 80: Puerto Rico.
Retrieving Results for Index 151: Pathein.
Retrieving Results for Index 163: Mount Isa.
Retrieving Results for Index 192: Delta del Tigre.
Retrieving Results for Index 213: Abu Dhabi.
Retrieving Results for Index 289: Gualeguay.
Retrieving Results for Index 305: Atar.
Retrieving Results for Index 324: Hīt.
Retrieving Results for Index 355: Upington.
Retrieving Results for Index 377: Dhanwār.


In [80]:
hotel_df

,Unnamed: 0,City,Lng,Lat,Temperature,Humidity,Wind Speed,Cloud Cover %,Country,Hotel Name
38,38,East London,27.91,-33.02,71.222,66,7.84,0,ZA,Prana Lodge
80,80,Puerto Rico,-55.02,-26.80,74.372,40,0.52,0,AR,Hotel Las Azaleas
151,151,Pathein,94.73,16.78,76.190,84,3.55,0,MM,Eskala Hotel and Resorts
163,163,Mount Isa,139.50,-20.73,71.600,30,4.10,0,AU,Burke & Wills Motel Mt Isa
192,192,Delta del Tigre,-56.39,-34.76,75.992,44,5.10,0,UY,Hotel Sofitel Montevideo Casino Carrasco and Spa
213,213,Abu Dhabi,54.37,24.47,77.000,73,3.10,0,AE,Emirates Palace
289,289,Gualeguay,-59.31,-33.14,74.462,46,2.47,0,AR,Estancia La Esperanza
305,305,Atar,-13.05,20.52,74.984,19,3.72,0,MR,Odar kanawal
324,324,Hīt,42.83,33.64,75.128,30,1.45,0,IQ,فندق روز بلازا السياحي _ Hotel Roz Blaza Tourist
355,355,Upington,21.26,-28.45,73.400,35,2.60,0,ZA,Oranjerus Resort


In [83]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [84]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))